<a href="https://www.kaggle.com/code/hatemamine/ledgarlexglue-preprocess-and-generate-dataset?scriptVersionId=107106489" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#!pip install keybert --quiet
#from keybert import KeyBERT
from datasets import load_dataset
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas 
import numpy as np 
import re
import matplotlib
import seaborn as sb 
import time
from datasets import load_dataset

In [ ]:
#get the number of words 
def getNumberOfWords(df):
    maxl=[]
    for i in range(len(df)):
        x= df["ATexte"][i].split()
        if len(x)< 3: print(i)
        maxl.append(len(x))
    print("Max value element : ", max(maxl))
    print("Min value element : ", min(maxl))
    return maxl

In [ ]:
def plot_NumberOfWord(maxl, dataset_name):
    fig = plt.figure()
    ax1 = plt.axes()
    ax1.set_ylabel('number of words')
    ax1.set_title('number of words for each instance '+dataset_name)
    ind = np.arange(len(maxl))
    line, =ax1.plot(ind,maxl)
    plt.axhline(y=512, color="red", linestyle="--")
    fig.savefig(dataset_name+".png")
    plt.show()

In [ ]:
def plotNumberSampleforeachCLASS(df, dataset_name):
    plot_=sb.countplot(df.MotsCles)
    new_ticks = [i.get_text() for i in plot_.get_xticklabels()]
    matplotlib.pyplot.xticks(range(0, len(new_ticks), 10), new_ticks[::10], rotation='vertical')
    matplotlib.pyplot.xlabel("Number of sample for each class Distribution "+dataset_name)
    matplotlib.pyplot.savefig(dataset_name+'distribution.png')
    plt.show()

In [ ]:
dataset = load_dataset("lex_glue", "ledgar")
DATASET_NAME='LEDGAR'
for x in dataset.keys():
    #get the train dataset
    dict = {'ATexte': dataset[x]["text"], 'MotsCles': dataset[x]["label"]}
    df = pandas.DataFrame(dict)
    df["ATexte"]= df["ATexte"].str.lower()
    for i in range(len(df)):
        df["ATexte"][i] = re.sub(r'[^a-z ]', '', df["ATexte"][i])
    print(x,len(df),"sample")
    plotNumberSampleforeachCLASS(df, DATASET_NAME+x)
    plot_NumberOfWord(getNumberOfWords(df), DATASET_NAME+x)
    df.to_csv("/kaggle/working/"+DATASET_NAME+x+".csv")
    #df = PreprocessLongTextwithKeyBERT(df, 3)
    #df = PreprocessLongTextwithKeyBERT(df, 1)
    #plot_NumberOfWord(getNumberOfWords(df), DATASET_NAME+x+"KeyBERT")
    #df.to_csv("/kaggle/working/"+DATASET_NAME+x+"KeyBERT.csv")

In [ ]:
df=pandas.read_csv("./LEDGARtrain.csv")
print(df.head())

In [ ]:
import random
random.seed(1)
text1=[]
text2=[]
correlated=[]
labels=[]
# 
for i in range (len(df)):
    row0 =df.iloc[i]
    # we exclud label 0 and 1 and 2 from the generated dataset for training 
    if row0["MotsCles"]==0 or row0["MotsCles"]==1 or row0["MotsCles"]==2:continue
    x=0
    y=0
    while x < 5:
        ind = random.randint(0, len(df)-1)
        if i ==ind : continue
        row =df.iloc[ind]
        if row0["MotsCles"]==row["MotsCles"]:
            text1.append(row0["ATexte"])
            text2.append(row["ATexte"])
            correlated.append(1)
            labels.append(str(row0["MotsCles"])+"=="+str(row["MotsCles"]))
            x=x+1
            
        
    while y < 5:
        ind = random.randint(0, len(df)-1)
        if i ==ind : continue
        row =df.iloc[ind]
        if row0["MotsCles"]!=row["MotsCles"]:
            text1.append(row0["ATexte"])
            text2.append(row["ATexte"])
            correlated.append(0)
            labels.append(str(row0["MotsCles"])+"!="+str(row["MotsCles"]))
            y=y+1
    #break
dict = {'ATexte':text1 , 'MotsCles':text2, 'correlated':correlated, 'labels':labels } 
df7 = pandas.DataFrame(dict)              
print(len(df7))
#print(df7["MotsCles"][0])
#print(df7["ATexte"][0])
print(df7.head())
#df7.to_csv("/kaggle/working/lexglueledgarTronsformed.csv")

In [ ]:
!pip install sentence-transformers --quiet
train, validate = np.split(df7.sample(frac=1, random_state=42),[int(.9*len(df7))])
#print(len(df7))
#print(len(train))
#print(len(validate))
#print(train.head())
from sentence_transformers import InputExample
dev_samples=[]
train_samples=[]
for i in range(len(train)):
        row =train.iloc[i]
        #As we want to get symmetric scores, i.e. CrossEncoder(A,B) = CrossEncoder(B,A), we pass both combinations to the train set
        train_samples.append(InputExample(texts=[row["ATexte"], row["MotsCles"]], label=float(row["correlated"])))
        #break
for i in range(len(validate)):
        row =validate.iloc[i]
        dev_samples.append(InputExample(texts=[row["ATexte"], row["MotsCles"]], label=float(row["correlated"])))
        #break
#print(train_samples[:10])

In [ ]:
import pickle
import torch
lexglueLedgar = {'train_samples':train_samples, 'dev_samples':dev_samples}
with open('/kaggle/working/lexglueLedgar012.pickle', 'wb') as handle:
    pickle.dump(lexglueLedgar, handle, protocol=pickle.HIGHEST_PROTOCOL)